In [15]:
import numpy as np
from scipy.optimize import minimize  #optimize函数用于求解非线性规划问题,minimize函数是其中的一个子函数

# 目标函数接受一个长度为2的一维数组,(代表两个变量x1,x2),计算并返回目标函数值
def fun1(x):
    return x[0]**2+x[1]**2-x[0]*x[1]-2*x[0]-5*x[1]

# 第一个非线性约束条件函数接受变量x,返回对应约束条件的值
def nonlcon1(x):
    return -(x[0]-1)**2+x[1]

# 第二个非线性约束条件函数,接受变量x,返回对应约束条件的值
def nonlcon2(x):
    return 2*x[0]-3*x[1]+6

In [16]:
# 例题一的求解
# 初始值设定,设置为一个二维的numpy数组,初始化为[0,0]
x0=np.array([0,0])

# 使用默认算法(内点法类似的算法,在scipy中会根据实际情况选择合适的)
res=minimize(fun1,x0,
             constraints=({'type': 'ineq', 'fun': nonlcon1},
                          {'type': 'ineq', 'fun': nonlcon2}),
             bounds=None,tol=None,options=None,args=())
print("默认算法(类似内点法)求解:")
print('最优解:',res.x)
print('最优值:',res.fun)



默认算法(类似内点法)求解:
最优解: [2.99999983 3.99999984]
最优值: -12.999999999999973


In [17]:
#使用SQP算法求解:
res_SQP=minimize(fun1,x0,method='SLSQP',
             constraints=({'type': 'ineq', 'fun': nonlcon1},
                          {'type': 'ineq', 'fun': nonlcon2}),
             bounds=None,tol=None,options=None,args=())
print("SQP算法求解:")
print('最优解:',res_SQP.x)
print('最优值:',res_SQP.fun)

SQP算法求解:
最优解: [2.99999983 3.99999984]
最优值: -12.999999999999973


In [18]:
# 利用蒙特卡洛算法
n=1000000
x1=np.random.uniform(-100,100,n)
x2=np.random.uniform(-100,100,n)
fmin=1000000
for i in range(n):
    x=np.array([x1[i],x2[i]])
    # 后续的约束条件判断和结果处理逻辑保持不变
    if nonlcon1(x)>=0 and nonlcon2(x)>=0:
        result=fun1(x)
        if result<fmin:
            fmin=result
            x0=x
print("蒙特卡洛算法选取初始值为:",x0)


#使用找到的初始值再次进行优化求解:
res_final=minimize(fun1,x0,method='SLSQP',
                   constraints=({'type': 'ineq', 'fun': nonlcon1},
                                {'type': 'ineq', 'fun': nonlcon2}),
                    bounds=None,tol=None,options=None,args=())

print("基于蒙特卡洛算法优化结果:")
print("x=",res_final.x)
print("fun1(x)=",res_final.fun)

蒙特卡洛算法选取初始值为: [2.85784209 3.79239326]
基于蒙特卡洛算法优化结果:
x= [3.00000014 4.00000009]
fun1(x)= -12.999999999999988


In [22]:
import numpy as np
from scipy.optimize import linprog, minimize, LinearConstraint, Bounds

# 第一问：线性规划
a = np.array([1.25, 8.75, 0.5, 5.75, 3, 7.25])
b = np.array([1.25, 0.75, 4.75, 5, 6.5, 7.75])
x_temp = np.array([5, 2])
y_temp = np.array([1, 7])
d = np.array([3, 5, 4, 7, 6, 11])

# 计算目标函数系数
l = np.zeros((6, 2))
for i in range(6):
    for j in range(2):
        l[i, j] = np.sqrt((x_temp[j] - a[i])**2 + (y_temp[j] - b[i])**2)
f = np.concatenate([l[:, 0], l[:, 1]])

# 不等式约束
A_ub = np.zeros((2, 12))
A_ub[0, :6] = 1
A_ub[1, 6:] = 1
b_ub = np.array([20, 20])

# 等式约束
A_eq = np.hstack([np.eye(6), np.eye(6)])
b_eq = d.copy()

# 变量边界
bounds = [(0, None)] * 12

# 求解线性规划
result_q1 = linprog(f, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
print("第一问结果：")
print("目标函数值：", result_q1.fun)
print("最优解：\n", result_q1.x)

# 第二问：非线性规划
def objective(x):
    x_trans = x[:12]
    x1, y1, x2, y2 = x[12], x[13], x[14], x[15]
    total = 0.0
    for i in range(6):
        dist1 = np.sqrt((x1 - a[i])**2 + (y1 - b[i])**2)
        total += x_trans[i] * dist1
        dist2 = np.sqrt((x2 - a[i])**2 + (y2 - b[i])**2)
        total += x_trans[i+6] * dist2
    return total

# 线性约束
A_ub_q2 = np.zeros((2, 16))
A_ub_q2[0, :6] = 1
A_ub_q2[1, 6:12] = 1
linear_ineq = LinearConstraint(A_ub_q2, ub=[20, 20])

A_eq_q2 = np.zeros((6, 16))
for i in range(6):
    A_eq_q2[i, i] = 1
    A_eq_q2[i, i+6] = 1
linear_eq = LinearConstraint(A_eq_q2, lb=d, ub=d)

# 变量边界
bounds_q2 = Bounds([0]*12 + [-np.inf]*4, [np.inf]*16)

# 初始值
x0_q2 = np.concatenate([result_q1.x, [5, 1, 2, 7]])

# 求解非线性规划
result_q2 = minimize(objective, x0_q2, method='SLSQP',
                     constraints=[linear_ineq, linear_eq],
                     bounds=bounds_q2)
print("\n第二问结果（初始值为第一问解）：")
print("目标函数值：", result_q2.fun)
print("最优解：\n", result_q2.x)

# 蒙特卡洛采样生成初始值
n_samples = 100000
best_val = np.inf
best_x = None

for _ in range(n_samples):
    x_trans = np.zeros(12)
    for i in range(6):
        x1_i = np.random.uniform(0, d[i])
        x_trans[i] = x1_i
        x_trans[i+6] = d[i] - x1_i
    sum1 = np.sum(x_trans[:6])
    sum2 = np.sum(x_trans[6:12])
    if sum1 > 20 or sum2 > 20:
        continue
    xy = np.random.uniform(0, 10, size=4)
    x_sample = np.concatenate([x_trans, xy])
    current_val = objective(x_sample)
    if current_val < best_val:
        best_val = current_val
        best_x = x_sample.copy()

print("\n蒙特卡洛最佳初始值的目标函数值：", best_val)

# 使用蒙特卡洛结果求解
if best_x is not None:
    result_q3 = minimize(objective, best_x, method='SLSQP',
                         constraints=[linear_ineq, linear_eq],
                         bounds=bounds_q2)
    print("\n第二问结果（蒙特卡洛初始值）：")
    print("目标函数值：", result_q3.fun)
    print("最优解：\n", result_q3.x)
else:
    print("未找到有效的蒙特卡洛初始解")

第一问结果：
目标函数值： 136.22751988318154
最优解：
 [ 3.  5.  0.  7.  0.  1.  0.  0.  4.  0.  6. 10.]

第二问结果（初始值为第一问解）：
目标函数值： 89.88348558442206
最优解：
 [3.00000000e+00 5.00000000e+00 4.00000000e+00 7.00000000e+00
 1.00000000e+00 1.25529910e-13 3.52086600e-14 2.25103623e-14
 5.03340191e-13 3.48726314e-15 5.00000000e+00 1.10000000e+01
 5.69416437e+00 4.92570261e+00 7.24999994e+00 7.74999997e+00]

蒙特卡洛最佳初始值的目标函数值： 104.67081197849339

第二问结果（蒙特卡洛初始值）：
目标函数值： 85.26604397684747
最优解：
 [3.00000000e+00 5.62667896e-13 4.00000000e+00 7.00000000e+00
 6.00000000e+00 1.68507403e-14 1.90653414e-14 5.00000000e+00
 3.80753636e-14 4.51027290e-13 1.41266492e-13 1.10000000e+01
 3.25492591e+00 5.65221438e+00 7.25000007e+00 7.74999949e+00]
